In [1]:
import pandas as pd #판다스 패키지 불러오기
import lightgbm as lgb

In [2]:
# 데이터 불러오기
train = pd.read_csv('/content/movies_train.csv')
test = pd.read_csv('/content/movies_test.csv')
submission = pd.read_csv('/content/submission.csv')

In [3]:
print(train.shape)
print(test.shape)
print(submission.shape)

(600, 12)
(243, 11)
(243, 2)


In [4]:
pd.options.display.float_format = '{:.1f}'.format

In [5]:
train[['genre', 'box_off_num']].groupby('genre').mean().sort_values('box_off_num')

,box_off_num
genre,
뮤지컬,6627.0
다큐멘터리,67172.3
서스펜스,82611.0
애니메이션,181926.7
멜로/로맨스,425968.0
미스터리,527548.2
공포,590832.5
드라마,625689.8
코미디,1193914.0


In [6]:
pd.reset_option('display.float_format')

In [7]:
train.corr()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
time,1.000000,0.264675,0.306727,0.623205,0.114153,0.441452
dir_prev_bfnum,0.264675,1.000000,0.131822,0.323521,0.083818,0.283184
dir_prev_num,0.306727,0.131822,1.000000,0.450706,0.014006,0.259674
num_staff,0.623205,0.323521,0.450706,1.000000,0.077871,0.544265
num_actor,0.114153,0.083818,0.014006,0.077871,1.000000,0.111179
box_off_num,0.441452,0.283184,0.259674,0.544265,0.111179,1.000000


In [8]:
train['dir_prev_bfnum'].fillna(0, inplace = True)
test[test['dir_prev_bfnum'].isna()]['dir_prev_num'].sum()

0

In [9]:
test['dir_prev_bfnum'].fillna(0, inplace = True)

In [10]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)

In [11]:
features = ['time', 'dir_prev_num', 'num_staff', 'num_actor']
target = ['box_off_num']

In [12]:
X_train, X_test, y_train = train[features], test[features], train[target]

In [13]:
model.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=777, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [14]:
singleLGBM = submission.copy()

In [15]:
singleLGBM.head()

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0


In [16]:
singleLGBM['box_off_num'] = model.predict(X_test)

In [48]:
singleLGBM

,title,box_off_num
0,용서는 없다,2.817995e+06
1,아빠가 여자를 좋아해,3.753772e+05
2,하모니,-5.693243e+05
3,의형제,1.581189e+06
4,평행 이론,-5.277806e+05
...,...,...
238,해에게서 소년에게,5.007844e+05
239,울보 권투부,1.013858e+06
240,어떤살인,1.682068e+06
241,말하지 못한 비밀,3.002163e+05


In [17]:
from sklearn.model_selection import KFold

In [18]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=777)

In [19]:
for train_idx, val_idx in k_fold.split(X_train):
    print(len(train_idx), len(val_idx))
    break

480 120


In [20]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)

models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_t = X_train.iloc[train_idx]
    y_t = y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set=(x_val, y_val), early_stopping_rounds=100, verbose = 100))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 2.70572e+12
Early stopping, best iteration is:
[6]	valid_0's l2: 2.45438e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 3.90847e+12
Early stopping, best iteration is:
[33]	valid_0's l2: 3.72825e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 3.50344e+12
Early stopping, best iteration is:
[8]	valid_0's l2: 2.58737e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.45977e+12
Early stopping, best iteration is:
[11]	valid_0's l2: 1.26226e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.77214e+12
Early stopping, best iteration is:
[22]	valid_0's l2: 1.57631e+12


In [21]:
models

[LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=777, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=777, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
  

In [22]:
preds = []
for model in models:
    preds.append(model.predict(X_test))
len(preds)

5

In [23]:
kfoldLightGBM = submission.copy()

In [24]:
import numpy as np

In [25]:
kfoldLightGBM['box_off_num'] = np.mean(preds, axis = 0)

In [27]:
kfoldLightGBM.to_csv('/content/kfoldLightGBM.csv', index = False)

In [28]:
kfoldLightGBM

,title,box_off_num
0,용서는 없다,3.367422e+06
1,아빠가 여자를 좋아해,9.611389e+05
2,하모니,1.097930e+06
3,의형제,2.097271e+06
4,평행 이론,7.814763e+05
...,...,...
238,해에게서 소년에게,1.770958e+05
239,울보 권투부,2.102555e+05
240,어떤살인,3.605476e+05
241,말하지 못한 비밀,1.815082e+05


In [29]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['genre'] = le.fit_transform(train['genre'])

In [30]:
test['genre'] = le.transform(test['genre'])

In [55]:
train['release_time']=train.release_time.apply(pd.to_datetime)
train['month'] = train.release_time.apply(lambda x : x.month)
test['release_time'] = test.release_time.apply(pd.to_datetime)
test['month'] = test.release_time.apply(lambda x : x.month)

train.drop(['release_time'],axis=1, inplace=True)
test.drop(['release_time'],axis=1, inplace=True)


In [75]:
train['is_adult'] = [ 1 if train.iloc[x, 5] != '청소년 관람불가' else 0 for x in range(0,600)]
test['is_adult'] = [ 1 if test.iloc[x, 4] != '청소년 관람불가' else 0 for x in range(0,test.shape[0])]


In [94]:
train['genre'] = le.inverse_transform(train['genre'])
test['genre'] = le.inverse_transform(test['genre'])

train.head(5)

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,month,is_adult
0,개들의 전쟁,롯데엔터테인먼트,액션,96,청소년 관람불가,조병옥,0.00,0,91,2,23398,11,1
1,내부자들,(주)쇼박스,느와르,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,11,1
2,은밀하게 위대하게,(주)쇼박스,액션,123,15세 관람가,장철수,220775.25,4,343,4,6959083,6,1
3,나는 공무원이다,(주)NEW,코미디,101,전체 관람가,구자홍,23894.00,2,20,6,217866,7,1
4,불량남녀,쇼박스(주)미디어플렉스,코미디,108,15세 관람가,신근호,1.00,1,251,2,483387,11,1


In [95]:
# 데이터에서 장르,관람등급 피처만 추출해서 데이터프레임 만들기
train_sam = train[['genre','screening_rat']]
test_sam = test[['genre','screening_rat']]

# 해당 데이터 프레임에 있는 피처들로 원핫인코딩
train_sam = pd.get_dummies(train_sam)
test_sam = pd.get_dummies(test_sam)

# 기존 트레인, 테스트 데이터에서 장르, 관람등급 피처 삭제
train.drop(['genre','screening_rat'],axis=1, inplace=True)
test.drop(['genre','screening_rat'],axis=1, inplace=True)

#원핫 인코딩 된걸 기존 트레인,테스트 데이터(장르 관람등급 삭제된 버전)와 병합해서 새 데이터 만들기
train = pd.concat([train,train_sam],axis=1)
test = pd.concat([test,test_sam],axis=1)


In [176]:
df=pd.concat([train, test]).reset_index(drop=True)

In [177]:
df['distributor']=df['distributor'].str.replace(" ","")

for i in range(len(df)):
  if "CJ" in df.iloc[i]["distributor"] or "CGV" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="CJ"
  elif "롯데" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="롯데엔터테인먼트"
  elif "마운틴" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="마운틴픽쳐스"
  elif "쇼박스" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="쇼박스"
  elif "NEW" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="NEW"
  elif "시네마달" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="시네마달"
  elif "꼴라쥬" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="무비꼴라쥬"
  elif "리틀" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="리틀빅픽처스"
  elif "스폰지" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="스폰지이엔티"
  elif "메가박스" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="메가박스"
  elif "싸이더스" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="싸이더스"
  elif "타임" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="타임스토리"
  elif "드림팩트" in df.iloc[i]["distributor"]:
    df.loc[i, "distributor"]="드림팩트엔터테인먼트"

dis_cnt=pd.DataFrame(df["distributor"].value_counts())

df["distributor_weight"]=[0]*len(df)

for i in range(len(df)):
  dis=df.loc[i, "distributor"]
  if dis_cnt.loc[dis, "distributor"]<=2:
    df.loc[i, "distributor"]="소형배급사"
  else:
    df.loc[i, "distributor_weight"]=dis_cnt.loc[dis, "distributor"]/len(dis_cnt)

In [193]:
train['box_off_num']=np.log1p(train['box_off_num'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [194]:
train

,title,distributor,time,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,month,is_adult,genre_SF,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,genre_뮤지컬,genre_미스터리,genre_서스펜스,genre_애니메이션,genre_액션,genre_코미디,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가,distributor_weight,index
0,개들의 전쟁,롯데엔터테인먼트,96,조병옥,0.00,0,91,2,2.403376,11,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1.54,NaN
1,내부자들,쇼박스,130,우민호,1161602.50,2,387,3,2.819694,11,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.84,NaN
2,은밀하게 위대하게,쇼박스,123,장철수,220775.25,4,343,4,2.818730,6,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.84,NaN
3,나는 공무원이다,NEW,101,구자홍,23894.00,2,20,6,2.587135,7,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.96,NaN
4,불량남녀,쇼박스,108,신근호,1.00,1,251,2,2.645364,11,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.84,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,해무,NEW,111,심성보,3833.00,1,510,7,2.721574,8,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.96,NaN
596,파파로티,쇼박스,127,윤종찬,496061.00,1,286,6,2.731491,3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0.84,NaN
597,살인의 강,마운틴픽쳐스,99,김대현,0.00,0,123,4,2.176387,9,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.98,NaN
598,악의 연대기,CJ,102,백운학,0.00,0,431,4,2.747307,5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2.02,NaN


In [178]:
train = df.iloc[:600,:]
test = df.iloc[600:]
test = test.reset_index()

In [180]:
features = ['time', 'dir_prev_num', 'num_staff', 'num_actor', 'dir_prev_bfnum', 'genre_SF', 'genre_공포', 'genre_느와르', 'genre_다큐멘터리',
       'genre_드라마', 'genre_멜로/로맨스', 'genre_뮤지컬', 'genre_미스터리', 'genre_서스펜스',
       'genre_애니메이션', 'genre_액션', 'genre_코미디', 'screening_rat_12세 관람가',
       'screening_rat_15세 관람가', 'screening_rat_전체 관람가',
       'screening_rat_청소년 관람불가','distributor_weight']

In [195]:
X_train, X_test, y_train = train[features], test[features], train[target]

In [196]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)

models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_t = X_train.iloc[train_idx]
    y_t = y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set=(x_val, y_val), early_stopping_rounds=100, verbose = 100))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0472818
Early stopping, best iteration is:
[17]	valid_0's l2: 0.0431155
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0563593
Early stopping, best iteration is:
[58]	valid_0's l2: 0.0540234
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0440922
Early stopping, best iteration is:
[40]	valid_0's l2: 0.0427425
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0474461
Early stopping, best iteration is:
[39]	valid_0's l2: 0.0446334
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0343772
Early stopping, best iteration is:
[78]	valid_0's l2: 0.0329013


In [197]:
preds = []
for model in models:
    preds.append(model.predict(X_test))
len(preds)

5

In [184]:
feLightGBM = submission.copy()

In [134]:
feLightGBM['box_off_num'] = np.mean(preds, axis = 0)

In [104]:
feLightGBM.to_csv('/content/feLightGBM.csv', index = False)

In [105]:
from sklearn.model_selection import GridSearchCV

In [198]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)


params = {
    'learning_rate': [0.1, 0.01, 0.003],
    'min_child_samples': [20, 30]}

gs = GridSearchCV(estimator=model,
            param_grid=params,
            scoring='neg_mean_squared_error',
            cv = k_fold)

In [199]:
gs.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=777, shuffle=True),
             error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=1000, n_jobs=-1,
                                     num_leaves=31, objective=None,
                                     random_state=777, reg_alpha=0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.01, 0.003],
                    

In [200]:
gs.best_params_

{'learning_rate': 0.003, 'min_child_samples': 20}

In [201]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000, learning_rate= 0.003, min_child_samples=30)

models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_t = X_train.iloc[train_idx]
    y_t = y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set=(x_val, y_val), early_stopping_rounds=100, verbose = 100))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0736141
[200]	valid_0's l2: 0.0595921
[300]	valid_0's l2: 0.0521671
[400]	valid_0's l2: 0.0482822
[500]	valid_0's l2: 0.046029
[600]	valid_0's l2: 0.0447348
[700]	valid_0's l2: 0.0438266
[800]	valid_0's l2: 0.0434482
[900]	valid_0's l2: 0.0431835
[1000]	valid_0's l2: 0.0428976
Did not meet early stopping. Best iteration is:
[981]	valid_0's l2: 0.0428696
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0976973
[200]	valid_0's l2: 0.0823118
[300]	valid_0's l2: 0.072904
[400]	valid_0's l2: 0.0670721
[500]	valid_0's l2: 0.0631709
[600]	valid_0's l2: 0.0607747
[700]	valid_0's l2: 0.0592049
[800]	valid_0's l2: 0.0585422
[900]	valid_0's l2: 0.0575849
[1000]	valid_0's l2: 0.056628
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 0.056628
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0838403
[200]	valid_0's l2: 0.0685

In [202]:
preds = []
for model in models:
    preds.append(model.predict(X_test))

In [203]:
gs.best_score_

-0.04439240688364449

In [191]:
rmse = np.sqrt(-1*gs.best_score_)
print(rmse)

1479160.9478981674


In [172]:
gslgbm = submission.copy()

In [173]:
gslgbm['box_off_num'] =  np.mean(preds, axis = 0)

In [174]:
gslgbm.to_csv('/content/gslgbm.csv', index = False)